In [6]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'

In [7]:
train = np.loadtxt(os.path.join(dir_, 'rating_matrix.csv'), delimiter=',')
pd_train = pd.read_pickle(os.path.join(dir_, 'train_normalized_to_rating_filter_track_20_user_100.pkl'))
train

array([[3.75, 3.5 , 4.75, ..., 0.  , 0.  , 0.  ],
       [0.  , 1.5 , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [8]:
num_user = len(train)
num_track = len(train[0])
print(num_user, num_track)

945 39088


In [9]:
def pearson_correlation(u1_index, u2_index):
        result = 0.0
        user1_data = train[u1_index]
        user2_data = train[u2_index]

        rx_avg = user_average_rating(user1_data)
        ry_avg = user_average_rating(user2_data)
        sxy = common_items(user1_data, user2_data)

        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for item in sxy:
            rxs = user1_data[item]
            rys = user2_data[item]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2, -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return len(sxy), result

def user_average_rating(u):
    avg_rating = 0.0
    for i in u:
        avg_rating += i
    avg_rating /= len(u) * 1.0
    return avg_rating

def common_items(u1, u2):
    result = []
    for i in range(num_track):
        if u1[i] > 0 and u2[i] > 0:
            result.append(i)
    return result

In [44]:
sim_table = pd.DataFrame(columns=['u1', 'u2', 'common', 'pearson']) 
for i in tqdm(range(num_user)):
    for j in range(i+1, num_user):
        c, p = pearson_correlation(i,j)
        if p != -2:
            sim_table = sim_table.append({'u1': i, 'u2': j, 'common': c, 'pearson': p}, ignore_index=True)
            sim_table = sim_table.append({'u1': j, 'u2': i, 'common': c, 'pearson': p}, ignore_index=True)

KeyboardInterrupt: 

In [39]:
sim_table_ = sim_table[sim_table['common']>10]
sim_table_ = sim_table_.sort_values(by=['pearson'],  ascending=False)
high_similarity = sim_table_[:int(len(sim_table_)*2/10)]
low_similarity =sim_table_[int(len(sim_table_)*8/10):]
high_similarity_v = sim_table_.iloc[int(len(sim_table_)*2/10)]['pearson']
low_similarity_v = sim_table_.iloc[int(len(sim_table_)*8/10)]['pearson']
print(high_similarity_v, low_similarity_v)

0.6993128968331564 0.4970825243256837


In [40]:
pairs = pd.DataFrame(columns=['uid', 'pair']) 
for i in range(num_user):
    pair = len(high_similarity[high_similarity['u1'] == i])
    pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
pairs = pairs.sort_values(by=['pair'],  ascending=True)

groups = []
group_size = 6
pairs = pairs[pairs['pair'] >= (group_size-1)]
p_status = len(pairs)
pbar = tqdm(total=len(pairs)) 
while len(pairs) >= group_size:
    group = []
    uid = pairs.iloc[0]['uid']
    pbar.update(p_status - len(pairs))
    p_status -= (p_status - len(pairs))
    pair = pairs.iloc[0]['pair']
    group.append(uid)
    if pair == (group_size-1):
        uid = high_similarity[high_similarity['u1'] == uid]
        for _, row in uid.iterrows():
            group.append(int(row['u2']))
        check_sim = True
        for i in range(1, group_size):
            for j in range(i+1, group_size):
                check = high_similarity[high_similarity['u1'] == group[i]]
                check = check[check['u2'] == group[j]]
                if len(check) == 0:
                    check_sim = False
        if check_sim:
#             print(group)
            groups.append(group)
            for u in group:
                high_similarity = high_similarity[high_similarity['u1'] != u]
                high_similarity = high_similarity[high_similarity['u2'] != u]
        else:
            high_similarity = high_similarity[high_similarity['u1'] != group[0]]
            high_similarity = high_similarity[high_similarity['u2'] != group[0]]
        pairs = pd.DataFrame(columns=['uid', 'pair']) 
        for i in range(num_user):
            pair = len(high_similarity[high_similarity['u1'] == i])
            pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
        pairs = pairs.sort_values(by=['pair'],  ascending=True)
        pairs = pairs[pairs['pair'] >= (group_size-1)]
    else:
        uid = high_similarity[high_similarity['u1'] == uid]
        avaliable_uid = []
        for _, row in pairs.iterrows():
            if len(uid[uid['u2'] == row['uid']]) > 0:
                avaliable_uid.append(int(row['uid']))
                
        pointer = []
        for i in range(group_size-1):
            pointer.append(i)

        while pointer[0] < (len(avaliable_uid) - (group_size - 1) + 2) and len(avaliable_uid) >= (group_size-1):
            group_ = group.copy()
            for i in range(group_size-1):
                group_.append(avaliable_uid[pointer[i]])
            check_sim = True
            for i in range(1, group_size):
                for j in range(i+1, group_size):
                    check = high_similarity[high_similarity['u1'] == group_[i]]
                    check = check[check['u2'] == group_[j]]
                    if len(check) == 0:
                        check_sim = False
            if check_sim:
#                 print('_', group_)
                groups.append(group_)
                for u in group_:
                    high_similarity = high_similarity[high_similarity['u1'] != u]
                    high_similarity = high_similarity[high_similarity['u2'] != u]
                break
            else:
                p = 0
                added = False
                while not added:
                    if (group_size-2) - p < 0:
                        pointer[0] = (len(avaliable_uid) - (group_size - 1) + 2)
                        break
                    if pointer[(group_size-2) - p] < (len(avaliable_uid) - 1 - p):
                        pointer[(group_size-2) - p] += 1
                        while p > 0:
                            p -= 1
                            if (group_size-2) - p != 0:
                                pointer[(group_size-2) - p] = pointer[(group_size-2) - p - 1] + 1
                        added = True
                    else:
                        p += 1
        if pointer[0] == (len(avaliable_uid) - (group_size - 1) + 2) or len(avaliable_uid) < (group_size-1):
            high_similarity = high_similarity[high_similarity['u1'] != group[0]]
            high_similarity = high_similarity[high_similarity['u2'] != group[0]]
        pairs = pd.DataFrame(columns=['uid', 'pair']) 
        for i in range(num_user):
            pair = len(high_similarity[high_similarity['u1']==i])
            pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
        pairs = pairs.sort_values(by=['pair'],  ascending=True)
        pairs = pairs[pairs['pair'] >= (group_size-1)]
groups

[[930, 485, 738, 730, 491, 198],
 [96, 24, 853, 1, 450, 514],
 [810, 917, 646, 278, 154, 690],
 [397, 622, 86, 749, 820, 90],
 [213, 216, 331, 226, 125, 634],
 [170, 259, 319, 173, 174, 378],
 [492, 363, 288, 788, 939, 747],
 [606, 137, 395, 557, 612, 65],
 [486, 804, 527, 631, 400, 70],
 [203, 214, 32, 85, 569, 671],
 [841, 625, 374, 101, 25, 789],
 [47, 910, 272, 687, 522, 906],
 [863, 812, 94, 907, 670, 92],
 [317, 626, 227, 150, 401, 466],
 [534, 196, 566, 141, 682, 867],
 [809, 290, 756, 711, 408, 760],
 [677, 821, 840, 245, 133, 118],
 [200, 175, 727, 279, 740, 675],
 [221, 474, 922, 772, 803, 575],
 [315, 182, 714, 269, 584, 937],
 [326, 136, 667, 413, 434, 456],
 [335, 798, 73, 285, 183, 879],
 [659, 281, 618, 436, 838, 877],
 [237, 912, 309, 872, 899, 4],
 [379, 316, 501, 280, 589, 835],
 [778, 267, 555, 579, 562, 201],
 [652, 617, 321, 289, 674, 262],
 [733, 702, 639, 577, 82, 439],
 [126, 5, 823, 843, 53, 896],
 [819, 298, 941, 665, 454, 785],
 [880, 759, 146, 83, 668, 54],


In [41]:
for group in groups:
    if len(group) != group_size:
        print('Group size error')
    for i in range(group_size):
        for j in range(i+1, group_size):
            u1 = group[i]
            u2 = group[j]
            c, p = pearson_correlation(u1, u2)
            if p < high_similarity_v:
                print('Similarity error', u1, u2, p)
user_status = np.zeros(num_user)
for group in groups:
    for u in group:
        user_status[u] += 1
for i in user_status:
    if i > 1:
        print('Depulicate error')

In [42]:
len(groups)

78

In [43]:
dir_ = '../data/groups/high/'
np.savetxt(os.path.join(dir_, 'group_' + str(group_size)  + '.csv'), groups, delimiter=",")